In [1]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

/home/hua2bv/miniconda3/envs/llm-tta/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
adaptive_model_name = "stabilityai/StableBeluga-7B"
adaptive_tokenizer = AutoTokenizer.from_pretrained(adaptive_model_name, trust_remote_code=True)
adaptive_model = AutoModelForCausalLM.from_pretrained(adaptive_model_name, torch_dtype=torch.float16, device_map="auto").eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.36s/it]


* "Every box of PLA from Hatchbox has been a delightful and stress-free experience for printing."
* "Every box of PLA from Hatchbox has been incredibly pleasant and stress-free for printing."
* "Every box of PLA from Hatchbox has been a delightful and stress-free experience for printing."
* "Each box of PLA from Hatchbox has been incredibly pleasant and stress-free for printing."

In [7]:
prompt_input = "Every box of PLA from Hatchbox has been so nice and worry free for printing."
prompt = """
### Instructions ###
The assistant is to paraphrase the input text.

### Input Text ###
Now paraphrase {{{"<style_input>"}}}.

Return the text in the format: {{{Paraphased Text}}}.

### Paraphased Text ###
Paraphased Input Text: {{{### Assistant:""".replace("<style_input>", prompt_input)

tokenized_prompt = adaptive_tokenizer.encode(prompt, return_tensors="pt").to(adaptive_model.device)
outputs = adaptive_model.generate(
                tokenized_prompt,
                max_new_tokens=1000,
                return_dict_in_generate=True,
                num_return_sequences=4,
                early_stopping=True,
                num_beam_groups=4,
                num_beams=4,
                diversity_penalty=0.5,
                # temperature=0.9,
                # do_sample=True
            )

print(outputs)

GenerateBeamDecoderOnlyOutput(sequences=tensor([[    1, 29871,    13,  2277, 29937,  2799,   582,  1953,   835,    13,
          1576, 20255,   338,   304,   610,   481,  1092,   559,   278,  1881,
          1426, 29889,    13,    13,  2277, 29937, 10567,  3992,   835,    13,
         10454,   610,   481,  1092,   559,   426,  6224, 29908, 26526,  3800,
           310,   349,  4375,   515,   379,   905,  1884,   756,  1063,   577,
          7575,   322, 15982,  3889,   363, 14010,  1213,   930,  1836,    13,
            13, 11609,   278,  1426,   297,   278,  3402, 29901,   426,  6224,
          2177, 12451,  1463,  3992,   930,  1836,    13,    13,  2277, 29937,
          1459, 12451,  1463,  3992,   835,    13,  2177, 12451,  1463, 10567,
          3992, 29901,   426,  6224,  2277, 29937,  4007, 22137, 29901,   376,
         26526,  3800,   310,   349,  4375,   515,   379,   905,  1884,   756,
          1063,   577,  7575,   322, 15982,  3889,   363, 14010,  1213,   930,
          18

In [8]:
outputs.sequences.shape

torch.Size([4, 164])

In [10]:
decoded_text = adaptive_tokenizer.batch_decode(outputs["sequences"])
for decoded_prompt in decoded_text:
    print("------------")
    print(decoded_prompt)
    # print(decoded_prompt.split("Assistant:")[1].strip().split("\n")[0])

------------
<s> 
### Instructions ###
The assistant is to paraphrase the input text.

### Input Text ###
Now paraphrase {{{"Every box of PLA from Hatchbox has been so nice and worry free for printing."}}}.

Return the text in the format: {{{Paraphased Text}}}.

### Paraphased Text ###
Paraphased Input Text: {{{### Assistant: "Every box of PLA from Hatchbox has been so nice and worry free for printing."}}}.

Return the text in the format: {{{Paraphased Text}}}.</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
------------
<s> 
### Instructions ###
The assistant is to paraphrase the input text.

### Input Text ###
Now paraphrase {{{"Every box of PLA from Hatchbox has been so nice and worry free for printing."}}}.

Return the text in the format: {{{Paraphased Text}}}.

### Paraphased Text ###
Paraphased Input Text: {{{### Assistant:

Every box of PLA from Hatchbox has been so nice and worry free for printing.